In [1]:
from typing import Callable, List, Tuple, Optional
import math

Vec = List[float]
Mat = List[List[float]]

# ============================
# Spausdinimas ir bazinė algebra
# ============================
def print_matrix(M: Mat, title: str = "", fmt: str = "{:12.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    for r in M:
        print(" ".join(fmt.format(v) for v in r))
    print()

def print_vector(v: Vec, title: str = "", fmt: str = "{:12.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    print(" ".join(fmt.format(x) for x in v))
    print()

def norm2(v: Vec) -> float:
    return math.sqrt(sum(x*x for x in v))

def vec_add(a: Vec, b: Vec) -> Vec:
    return [a[i] + b[i] for i in range(len(a))]

def vec_sub(a: Vec, b: Vec) -> Vec:
    return [a[i] - b[i] for i in range(len(a))]

def vec_scale(a: Vec, s: float) -> Vec:
    return [s*ai for ai in a]

def dot(a: Vec, b: Vec) -> float:
    return sum(a[i]*b[i] for i in range(len(a)))

def matvec(A: Mat, x: Vec) -> Vec:
    return [sum(A[i][j]*x[j] for j in range(len(x))) for i in range(len(A))]

def eye(n: int) -> Mat:
    return [[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]

def deepcopy(A: Mat) -> Mat:
    return [row[:] for row in A]

# ============================
# 0) SKAITINIS FUNKCIJOS GRADIENTAS
#    ∇Ψ(x) ≈ [(Ψ(x+he1)-Ψ(x))/h, ...]
# ============================
def numerical_gradient(Psi: Callable[[Vec], float], x: Vec, h: float = 1e-6, verbose: bool=True) -> Vec:
    fx = Psi(x)
    g = [0.0]*len(x)
    for j in range(len(x)):
        xh = x[:]
        xh[j] += h
        g[j] = (Psi(xh) - fx) / h
    if verbose:
        print_vector(x, "Gradiento skaičiavimas taške x")
        print_vector(g, "∇Ψ(x) (skaitinis įvertis)")
        print(f"||∇Ψ(x)||2 = {norm2(g):.12g}\n")
    return g

# ============================
# Atgalinio žingsnio (Armijo) paieška
# ============================
def backtracking(Psi: Callable[[Vec], float],
                 x: Vec, d: Vec, g: Vec,
                 alpha0: float = 1.0,
                 c: float = 1e-4, rho: float = 0.5,
                 verbose: bool=True) -> float:
    """
    Randa žingsnį α > 0 tenkinant Armijo sąlygą:
      Ψ(x + α d) ≤ Ψ(x) + c α g^T d
    """
    fx = Psi(x)
    gd = dot(g, d)
    alpha = alpha0
    k = 0
    while True:
        x_new = vec_add(x, vec_scale(d, alpha))
        f_new = Psi(x_new)
        if f_new <= fx + c*alpha*gd:
            if verbose:
                print(f"[Backtracking] α={alpha:.6g}, f_old={fx:.6g}, f_new={f_new:.6g}  ✓")
            return alpha
        if alpha < 1e-16:
            if verbose:
                print(f"[Backtracking] α nukrito iki {alpha:.1e} – grąžinu minimalų.")
            return alpha
        if verbose and k < 3:
            print(f"[Backtracking] α={alpha:.6g} netinka (f_new={f_new:.6g}) → mažinu")
        alpha *= rho
        k += 1

# ============================
# 1) MINIMIZAVIMAS PAGAL GRADIENTĄ (fiksuoto dydžio žingsnis kryptimi -ĝ)
#    x_{k+1} = x_k - s * ĝ(x_k), kur ĝ = ∇Ψ / ||∇Ψ||
# ============================
def gradient_method(Psi: Callable[[Vec], float],
                    x0: Vec,
                    step: float = 1e-1,
                    eps: float = 1e-8,
                    nitmax: int = 10_000,
                    verbose: bool=True) -> Vec:
    x = x0[:]
    f = Psi(x)
    g = numerical_gradient(Psi, x, verbose=False)
    print_vector(x, "Gradientinis: pradinis x(0)")
    print(f"Gradientinis: f(x(0))={f:.12g}, ||∇Ψ||2={norm2(g):.12g}, step={step}\n")

    for it in range(1, nitmax+1):
        g = numerical_gradient(Psi, x, verbose=False)  # skaitinis ∇F
        gnorm = norm2(g)
        if gnorm < eps:
            print(f"[Gradientinis] Sustojau: ||∇Ψ||2 < eps ({gnorm:.3e} < {eps}).")
            break
    
        d = vec_scale(g, -1.0)           # -∇F (ne normalizuotas)
        x_new = vec_add(x, vec_scale(d, step))
        f_old = f
        f_new = Psi(x_new)
        if verbose:
            print_vector(g, f"[Gradientinis] it={it}: ∇Ψ(x)")
            print(f"[Gradientinis] it={it}: f_old={f_old:.6g} → f_new={f_new:.6g} (step={step:.3g})")
    
        x, f = x_new, f_new
    
        if gnorm < eps or step < 1e-16:
            print(f"[Gradientinis] Sustojau: ||∇Ψ||2={gnorm:.3e}, step={step:.1e}")
            break


    print_vector(x, "Gradientinis: sprendinys x")
    print(f"Gradientinis: f(x)={Psi(x):.12g}, ||∇Ψ(x)||2={norm2(numerical_gradient(Psi, x, verbose=False)):.12g}\n")
    return x

# ============================
# 2) GREIČIAUSIO NUSILEIDIMO METODAS (su Armijo atgaliniu žingsniu)
#    x_{k+1} = x_k + α_k d_k, d_k = -∇Ψ(x_k)
# ============================
def steepest_descent(Psi: Callable[[Vec], float],
                     x0: Vec,
                     alpha0: float = 1.0,
                     eps: float = 1e-8,
                     nitmax: int = 10_000,
                     verbose: bool=True) -> Vec:
    x = x0[:]
    f = Psi(x)
    print_vector(x, "Greičiausio nusileidimo: pradinis x(0)")
    print(f"Greičiausio nusileidimo: f(x(0))={f:.12g}\n")

    for it in range(1, nitmax+1):
        g = numerical_gradient(Psi, x, verbose=False)
        gnorm = norm2(g)
        if gnorm < eps:
            print(f"[Steepest] Sustojau: ||∇Ψ||2 < eps ({gnorm:.3e} < {eps}).")
            break

        d = vec_scale(g, -1.0)                          # kryptis = -∇Ψ
        alpha = backtracking(Psi, x, d, g, alpha0=alpha0, verbose=verbose)
        x_new = vec_add(x, vec_scale(d, alpha))
        f_new = Psi(x_new)

        if verbose:
            print_vector(g,     f"[Steepest] it={it}: ∇Ψ(x)")
            print(f"[Steepest] it={it}: f_old={f:.6g} → f_new={f_new:.6g}, α={alpha:.6g}")

        x, f = x_new, f_new
        if alpha < 1e-16:
            print("[Steepest] Sustojau: α ~ 0.")
            break
        if gnorm < eps:
            break

    print_vector(x, "Greičiausio nusileidimo: sprendinys x")
    print(f"Greičiausio nusileidimo: f(x)={Psi(x):.12g}, ||∇Ψ(x)||2={norm2(numerical_gradient(Psi, x, verbose=False)):.12g}\n")
    return x

# ============================
# 3) KVAZI-GRADIENTO METODAS NLS atveju:
#    Ψ(x) = 1/2 ||f(x)||^2,  g(x) = J(x)^T f(x)
#    Approx. J ≈ A, atnaujinam Broideno (good) rang-1 formule:
#    A_{k+1} = A_k + ((y - A_k s) s^T) / (s^T s),  y = f(x_{k+1}) - f(x_k)
#    Kryptis d = - g = - A^T f, žingsnis – Armijo.
# ============================
def quasi_gradient_nls(f: Callable[[Vec], Vec],
                       x0: Vec,
                       A0: Optional[Mat] = None,     # pradinė Jakobio aproksimacija; jei None – skaitinė J(x0)
                       h_jac: float = 1e-6,
                       alpha0: float = 1.0,
                       eps: float = 1e-8,
                       nitmax: int = 1_000,
                       verbose: bool=True) -> Vec:
    def Psi(x: Vec) -> float:
        fx = f(x)
        return 0.5*dot(fx, fx)

    # pradžia
    x = x0[:]
    fx = f(x)
    m, n = len(fx), len(x)

    # jeigu A0 neduota – pasidarom skaitinį J(x0)
    if A0 is None:
        A = [[0.0]*n for _ in range(m)]
        f0 = fx[:]
        for j in range(n):
            xh = x[:]; xh[j] += h_jac
            df = [f(xh)[i] - f0[i] for i in range(m)]
            for i in range(m):
                A[i][j] = df[i]/h_jac
        if verbose:
            print_matrix(A, "Kvazi-gradiento: pradinė A0 (skaitinė J(x0))")
    else:
        A = deepcopy(A0)
        if verbose:
            print_matrix(A, "Kvazi-gradiento: pradinė A0 (duota)")

    print_vector(x,  "Kvazi-gradiento: x(0)")
    print_vector(fx, "Kvazi-gradiento: f(x(0))")
    print(f"Kvazi-gradiento: Ψ(x(0)) = {0.5*dot(fx,fx):.12g}\n")

    for it in range(1, nitmax+1):
        # g = A^T f
        g = [sum(A[i][j]*fx[i] for i in range(m)) for j in range(n)]
        gnorm = norm2(g)
        if gnorm < eps:
            print(f"[QG] Sustojau: ||g||2 < eps ({gnorm:.3e} < {eps}).")
            break

        d = vec_scale(g, -1.0)  # kryptis
        # Armijo žingsnis
        alpha = backtracking(Psi, x, d, g, alpha0=alpha0, verbose=verbose)
        s = vec_scale(d, alpha)
        x_new = vec_add(x, s)
        fx_new = f(x_new)

        if verbose:
            print_vector(g,     f"[QG] it={it}: g = A^T f")
            print(f"[QG] it={it}: Ψ_old={0.5*dot(fx,fx):.6g} → Ψ_new={0.5*dot(fx_new,fx_new):.6g}, α={alpha:.6g}")

        # Broideno (good) atnaujinimas A
        y = vec_sub(fx_new, fx)           # f(x_{k+1}) - f(x_k)
        sTs = dot(s, s)
        if sTs < 1e-30:
            print("[QG] s^T s ≈ 0 – praleidžiu A atnaujinimą.")
        else:
            As = matvec(A, s)
            diff = [y[i] - As[i] for i in range(m)]
            # A <- A + diff * s^T / (s^T s)
            for i in range(m):
                for j in range(n):
                    A[i][j] += diff[i]*s[j]/sTs
            if verbose:
                print_matrix(A, f"[QG] it={it}: atnaujinta A")

        x, fx = x_new, fx_new

        if norm2(g) < eps or alpha < 1e-16:
            print(f"[QG] Sustojau: ||g||2={norm2(g):.3e}, α={alpha:.1e}")
            break

    print_vector(x,  "Kvazi-gradiento: sprendinys x")
    print_vector(fx, "Kvazi-gradiento: f(x)")
    print(f"Kvazi-gradiento: Ψ(x) = {0.5*dot(fx,fx):.12g},  ||g||2 ≈ {norm2([sum(A[i][j]*fx[i] for i in range(m)) for j in range(n)]):.12g}\n")
    return x

# ============================
# MAIN — tik užkomentuoti paleidimai (atsikomentuok, ką nori matyti)
# ============================
def main():
    # --- Pavyzdinė skaliarinė funkcija Ψ(x) (2D) ---
    # "Bowl + ridge": minimumas ties (1, -2)
    def Psi(x: Vec) -> float:
        return x[0]**2 + (x[1] - 5)**2 + x[0]**2 * x[1]

    x0 = [1.0, 2.0]

    # --- 0) Skaitinis gradientas ---
    # g = numerical_gradient(Psi, x0, h=1e-6, verbose=True)

    # --- 1) Gradientinis metodas (fiksuotas žingsnio mastelis) ---
    x_g = gradient_method(Psi, x0, step=0.1, eps=1e-8, nitmax=1, verbose=True)

    # --- 2) Greičiausio nusileidimo metodas (Armijo) ---
    # x_sd = steepest_descent(Psi, x0, alpha0=1.0, eps=1e-8, nitmax=2000, verbose=True)

    # --- 3) Kvazi-gradiento metodas NLS atveju: Ψ(x)=1/2||f(x)||^2 ---
    # Paimkim NLS pavyzdį iš ankstesnių pratybų:
    # f1 = x1^2 + x2^2 - 2;  f2 = x1^2 - x2^2
    def fNLS(x: Vec) -> Vec:
        return [x[0]**2 + x[1]**2 - 2.0,
                x[0]**2 - x[1]**2]
    x0_nls = [-2.0, 0.5]
    # x_qg = quasi_gradient_nls(fNLS, x0_nls, A0=None, alpha0=1.0, eps=1e-10, nitmax=200, verbose=True)

    pass

main()



Gradientinis: pradinis x(0)
---------------------------
           1            2

Gradientinis: f(x(0))=12, ||∇Ψ||2=7.81025134106, step=0.1


[Gradientinis] it=1: ∇Ψ(x)
--------------------------
           6           -5

[Gradientinis] it=1: f_old=12 → f_new=6.81 (step=0.1)

Gradientinis: sprendinys x
--------------------------
         0.4          2.5

Gradientinis: f(x)=6.80999964383, ||∇Ψ(x)||2=5.59156525204

